# Draft Intro into Strings 


## Working with text data
## Text data types 

There are two ways to store text data in pandas:

1. object -dtype NumPy array.

1. StringDtype extension type.

We recommend using StringDtype to store text data.

Prior to pandas 1.0, object dtype was the only option. This was unfortunate for many reasons:

1. You can accidentally store a mixture of strings and non-strings in an object dtype array. It’s better to have a dedicated dtype.

1. object dtype breaks dtype-specific operations like DataFrame.select_dtypes(). There isn’t a clear way to select just text while excluding non-text but still object-dtype columns.

1. When reading code, the contents of an object dtype array is less clear than 'string'.

Currently, the performance of object dtype arrays of strings and arrays.StringArray are about the same. We expect future enhancements to significantly increase the performance and lower the memory overhead of StringArray.



Before we begin, let's check out our hardware setup by running the nvidia-smi command.

In [1]:
!nvidia-smi

Sat Mar 27 00:03:53 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 450.80.02    Driver Version: 450.80.02    CUDA Version: 11.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            On   | 00000000:00:1E.0 Off |                    0 |
| N/A   46C    P0    27W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
import pandas as pd; print('Pandas Version:', pd.__version__)
import numpy as np
import cupy as cp
import cudf; print('CuDF Version:', cudf.__version__)
import warnings
warnings.filterwarnings('ignore')


Pandas Version: 1.1.5
CuDF Version: 0.18.0


For backwards-compatibility, object dtype remains the default type we infer a list of strings to

In [3]:
# Pandas

pd.Series(["a", "b", "c"])

0    a
1    b
2    c
dtype: object

In [4]:
# cuDF

cudf.Series(["a", "b", "c"])


0    a
1    b
2    c
dtype: object

To explicitly request string dtype, specify the dtype

In [5]:
pd.Series(["a", "b", "c"], dtype="string")

0    a
1    b
2    c
dtype: string

In [6]:
cudf.Series(["a", "b", "c"], dtype="str")

0    a
1    b
2    c
dtype: object

Or astype after the Series or DataFrame is created

In [7]:
pandasSeries = pd.Series(["a", "b", "c"])
pandasSeries.astype("string")
print(pandasSeries.astype("string"))
print(pandasSeries.str.repeat(2))
print(pandasSeries.str.count('n'))

0    a
1    b
2    c
dtype: string
0    aa
1    bb
2    cc
dtype: object
0    0
1    0
2    0
dtype: int64


In [8]:
cudfSeries = cudf.Series(["a", "b", "c"])
print(cudfSeries.astype("string"))
print(cudfSeries.repeat(2))
print(cudfSeries.str.count('n'))

0    a
1    b
2    c
dtype: object
0    a
0    a
1    b
1    b
2    c
2    c
dtype: object
0    0
1    0
2    0
dtype: int32


You can also use StringDtype/"string" as the dtype on non-string data and it will be converted to string dtype:

In [9]:
pandasSeries = pd.Series(["a", 2, np.nan], dtype="string")
pandasSeries
type(pandasSeries[1])

str

In [10]:
cudfSeries = cudf.Series(["a", 2, np.nan], dtype="str")
cudfSeries
type(cudfSeries[1])

str

or convert from existing pandas data:

In [11]:
pandasSeries = pd.Series([1, 2, np.nan], dtype="Int64")
pandasSeries
pandasSeries2 = pandasSeries.astype("string")
pandasSeries2
type(pandasSeries2[0])

str

In [12]:
cudfSeries1 = cudf.Series([1, 2, np.nan], dtype="int64")
cudfSeries
cudfSeries2 = cudfSeries1.astype("string")
cudfSeries2
type(cudfSeries2[0])

str

## Behavior differences

These are places where the behavior of StringDtype objects differ from object dtype

For StringDtype, string accessor methods that return numeric output will always return a nullable integer dtype, rather than either int or float dtype, depending on the presence of NA values. Methods returning boolean output will return a nullable boolean dtype.

In [13]:
pandasSeries = pd.Series(["a", None, "b"], dtype="string")
pandasSeries
pandasSeries.str.count("a")
pandasSeries.dropna().str.count("a")
print(pandasSeries.str.isnumeric())


0    False
1     <NA>
2    False
dtype: boolean


In [14]:
cudfSeries = cudf.Series(["a", None, "b"], dtype="str")
cudfSeries
cudfSeries.str.count("a")
cudfSeries.dropna().str.count("a")
print(cudfSeries.str.isnumeric())

0    False
1     <NA>
2    False
dtype: bool


Both outputs are Int64 dtype. Compare that with object-dtype

In [15]:
pandasSeries2 = pd.Series(["a", None, "b"], dtype="object")

pandasSeries2.str.count("a")

pandasSeries2.dropna().str.count("a")

0    1
2    0
dtype: int64

In [16]:
cudfSeries2 = cudf.Series(["a", None, "b"], dtype="object")

cudfSeries2.str.count("a")

cudfSeries2.dropna().str.count("a")

0    1
2    0
dtype: int32

When NA values are present, the output dtype is float64. Similarly for methods returning boolean values.

In [17]:
pandasSeries.str.isdigit()
pandasSeries.str.match("a")

0     True
1     <NA>
2    False
dtype: boolean

In [18]:
cudfSeries.str.isdigit()
cudfSeries.str.match("a")

0     True
1     <NA>
2    False
dtype: bool

Some string methods, like Series.str.decode() are not available on StringArray because StringArray only holds strings, not bytes.

In comparison operations, arrays.StringArray and Series backed by a StringArray will return an object with BooleanDtype, rather than a bool dtype object. Missing values in a StringArray will propagate in comparison operations, rather than always comparing unequal like numpy.nan.

Everything else that follows in the rest of this document applies equally to string and object dtype.


## String methods

Series and Index are equipped with a set of string processing methods that make it easy to operate on each element of the array. Perhaps most importantly, these methods exclude missing/NA values automatically. These are accessed via the str attribute and generally have names matching the equivalent (scalar) built-in string methods:

In [19]:
pandasSeries = pd.Series(
  ....:     ["A", "B", "C", "Aaba", "Baca", np.nan, "CABA", "dog", "cat"], dtype="string"
  ....: )
  ....: 

pandasSeries.str.lower()
print(pandasSeries.str.islower())

pandasSeries.str.upper()
print(pandasSeries.str.isupper())

pandasSeries.str.len()



0    False
1    False
2    False
3    False
4    False
5     <NA>
6    False
7     True
8     True
dtype: boolean
0     True
1     True
2     True
3    False
4    False
5     <NA>
6     True
7    False
8    False
dtype: boolean


0       1
1       1
2       1
3       4
4       4
5    <NA>
6       4
7       3
8       3
dtype: Int64

In [20]:
cudfSeries = cudf.Series(
  ....:     ["A", "B", "C", "Aaba", "Baca", np.nan, "CABA", "dog", "cat"], dtype="str"
  ....: )
  ....: 

cudfSeries.str.lower()
print(cudfSeries.str.islower())
cudfSeries.str.upper()
print(cudfSeries.str.islower())
cudfSeries.str.len()


0    False
1    False
2    False
3    False
4    False
5     <NA>
6    False
7     True
8     True
dtype: bool
0    False
1    False
2    False
3    False
4    False
5     <NA>
6    False
7     True
8     True
dtype: bool


0       1
1       1
2       1
3       4
4       4
5    <NA>
6       4
7       3
8       3
dtype: int32

In [21]:
pandasIdx = pd.Index([" jack", "jill ", " jesse ", "frank"])

pandasIdx.str.strip()


pandasIdx.str.lstrip()


pandasIdx.str.rstrip()


Index([' jack', 'jill', ' jesse', 'frank'], dtype='object')

In [22]:
cudfIdx = cudf.Index([" jack", "jill ", " jesse ", "frank"])

cudfIdx.str.strip()

cudfIdx.str.lstrip()

cudfIdx.str.rstrip()


StringIndex([' jack' 'jill' ' jesse' 'frank'], dtype='object')

The string methods on Index are especially useful for cleaning up or transforming DataFrame columns. For instance, you may have columns with leading or trailing whitespace:

In [23]:
pandasDataFrame = pd.DataFrame(np.random.randn(3, 2), columns=[" Column A ", " Column B "], index=range(3))
   
pandasDataFrame

,Column A,Column B
0,-1.064957,-0.558236
1,0.724952,-1.825184
2,-0.570135,0.808024


In [24]:
cudfDataFrame = cudf.DataFrame(np.random.randn(3, 2), columns=[" Column A ", " Column B "], index=range(3))
   
cudfDataFrame

,Column A,Column B
0,-2.114442,0.474351
1,-0.934478,0.856074
2,-1.559068,-0.291150


Since df.columns is an Index object, we can use the .str accessor

In [25]:
pandasDataFrame.columns.str.strip()

pandasDataFrame.columns.str.lower()

Index([' column a ', ' column b '], dtype='object')

In [26]:
cudfDataFrame.columns.str.strip()

cudfDataFrame.columns.str.lower()

Index([' column a ', ' column b '], dtype='object')

These string methods can then be used to clean up the columns as needed. Here we are removing leading and trailing whitespaces, lower casing all names, and replacing any remaining whitespaces with underscores:

In [27]:
pandasDataFrame.columns = pandasDataFrame.columns.str.strip().str.lower().str.replace(" ", "_")
pandasDataFrame

,column_a,column_b
0,-1.064957,-0.558236
1,0.724952,-1.825184
2,-0.570135,0.808024


In [28]:
cudfDataFrame.columns = cudfDataFrame.columns.str.strip().str.lower().str.replace(" ", "_")
cudfDataFrame

,column_a,column_b
0,-2.114442,0.474351
1,-0.934478,0.856074
2,-1.559068,-0.291150


## Splitting and replacing strings

Methods like split return a Series of lists:

In [29]:
pandasSeries3 = pd.Series(["a_b_c", "c_d_e", np.nan, "f_g_h"], dtype="string")
pandasSeries3.str.split("_")

0    [a, b, c]
1    [c, d, e]
2         <NA>
3    [f, g, h]
dtype: object

In [30]:
cudfSeries3 = cudf.Series(["a_b_c", "c_d_e", np.nan, "f_g_h"], dtype="str")
cudfSeries3.str.split("_")

0    [a, b, c]
1    [c, d, e]
2         None
3    [f, g, h]
dtype: list

It is easy to expand this to return a DataFrame using expand.

In [31]:
pandasSeries3.str.split("_", expand=True)

,0,1,2
0,a,b,c
1,c,d,e
2,<NA>,<NA>,<NA>
3,f,g,h


In [32]:
cudfSeries3.str.split("_", expand=True)

,0,1,2
0,a,b,c
1,c,d,e
2,<NA>,<NA>,<NA>
3,f,g,h


When original Series has StringDtype, the output columns will all be StringDtype as well.

It is also possible to limit the number of splits:

In [33]:
pandasSeries3.str.split("_", expand=True, n=1)

,0,1
0,a,b_c
1,c,d_e
2,<NA>,<NA>
3,f,g_h


In [34]:
cudfSeries3.str.split("_", expand=True, n=1)

,0,1
0,a,b_c
1,c,d_e
2,<NA>,<NA>
3,f,g_h


rsplit is similar to split except it works in the reverse direction, i.e., from the end of the string to the beginning of the string:

In [35]:
pandasSeries.str.rsplit("_", expand=True, n=1)

,0
0,A
1,B
2,C
3,Aaba
4,Baca
5,<NA>
6,CABA
7,dog
8,cat


In [36]:
cudfSeries3.str.rsplit("_", expand=True, n=1)

,0,1
0,a_b,c
1,c_d,e
2,<NA>,<NA>
3,f_g,h


replace optionally uses regular expressions:

In [37]:
pandasSeries4 = pd.Series(
   ....:     ["A", "B", "C", "Aaba", "Baca", "", np.nan, "CABA", "dog", "cat"],
   ....:     dtype="string",
   ....: )
   ....: 

pandasSeries4 

pandasSeries4.str.replace("^.a|dog", "XX-XX ", case=False, regex=True)

0           A
1           B
2           C
3    XX-XX ba
4    XX-XX ca
5            
6        <NA>
7    XX-XX BA
8      XX-XX 
9     XX-XX t
dtype: string

In [38]:
cudfSeries4 = cudf.Series(
   ....:     ["A", "B", "C", "Aaba", "Baca", "", np.nan, "CABA", "dog", "cat"],
   ....:     dtype="str",
   ....: )
   ....: 

cudfSeries4  
cudfSeries4.str.replace("^.a|dog", "XX-XX ",  regex=True)
# s4.str.replace("^.a|dog", "XX-XX ",  regex=True) case=False,

0           A
1           B
2           C
3    XX-XX ba
4    XX-XX ca
5            
6        <NA>
7        CABA
8      XX-XX 
9     XX-XX t
dtype: object

If you want literal replacement of a string (equivalent to str.replace()), you can set the optional regex parameter to False, rather than escaping each character. In this case both pat and repl must be strings:

In [39]:
pandasdollars = pd.Series(["12", "-$10", "$10,000"], dtype="string")

# These lines are equivalent
pandasdollars.str.replace(r"-\$", "-", regex=True)
pandasdollars.str.replace("-$", "-", regex=False)

0         12
1        -10
2    $10,000
dtype: string

In [40]:
cudfDollars = cudf.Series(["12", "-$10", "$10,000"], dtype="str")

# These lines are equivalent
cudfDollars.str.replace(r"-\$", "-", regex=True)
cudfDollars.str.replace("-$", "-", regex=False)

0         12
1        -10
2    $10,000
dtype: object

The replace method 

## Concatenation


There are several ways to concatenate a Series or Index, either with itself or others, all based on cat(), resp. Index.str.cat.

### Concatenating a single Series into a string

The content of a Series (or Index) can be concatenated:

In [41]:
pandasSeries = pd.Series(["a", "b", "c", "d"], dtype="string")

pandasSeries.str.cat(sep=",")

'a,b,c,d'

In [42]:
cudfSeries = cudf.Series(["a", "b", "c", "d"], dtype="str")

cudfSeries.str.cat(sep=",")

'a,b,c,d'

If not specified, the keyword sep for the separator defaults to the empty string, sep='':

In [43]:
pandasSeries.str.cat()

'abcd'

In [44]:
cudfSeries.str.cat()

'abcd'

By default, missing values are ignored. Using na_rep, they can be given a representation:

In [45]:
pandasSeriesB = pd.Series(["a", "b", np.nan, "d"], dtype="string")

pandasSeriesB.str.cat(sep=",")
pandasSeriesB.str.cat(sep=",", na_rep="-")

'a,b,-,d'

In [46]:
cudfSeriesB = cudf.Series(["a", "b", np.nan, "d"], dtype="str")

cudfSeriesB.str.cat(sep=",")
cudfSeriesB.str.cat(sep=",", na_rep="-")

'a,b,-,d'

## Concatenating a Series and something list-like into a Series

The first argument to cat() can be a list-like object, provided that it matches the length of the calling Series (or Index).

In [47]:
pandasSeries.str.cat(["A", "B", "C", "D"])

0    aA
1    bB
2    cC
3    dD
dtype: string

In [48]:
cudfSeries.str.cat(["A", "B", "C", "D"])

0    aA
1    bB
2    cC
3    dD
dtype: object

Missing values on either side will result in missing values in the result as well, unless na_rep is specified:

In [49]:
pandasSeries.str.cat(pandasSeriesB)
pandasSeries.str.cat(pandasSeriesB, na_rep="-")

0    aa
1    bb
2    c-
3    dd
dtype: string

In [50]:
cudfSeries.str.cat(cudfSeriesB)
cudfSeries.str.cat(cudfSeriesB, na_rep="-")

0    aa
1    bb
2    c-
3    dd
dtype: object

Indexing with .str

You can use [] notation to directly index by position locations. If you index past the end of the string, the result will be a NaN.

In [51]:
pandasSeries = pd.Series(["A", "B", "C", "Aaba", "Baca", np.nan, "CABA", "dog", "cat"], dtype="string")
   

pandasSeries.str[0]
pandasSeries.str[1]

0    <NA>
1    <NA>
2    <NA>
3       a
4       a
5    <NA>
6       A
7       o
8       a
dtype: string

In [52]:
cudfSeries = cudf.Series(["A", "B", "C", "Aaba", "Baca", np.nan, "CABA", "dog", "cat"], dtype="str")
   

cudfSeries.str[0]
cudfSeries.str[1]

0        
1        
2        
3       a
4       a
5    <NA>
6       A
7       o
8       a
dtype: object

## Extracting substrings

Extract first match in each subject (extract)

In [53]:
pd.Series(["a1", "b2", "c3"],dtype="string",).str.extract(r"([ab])(\d)", )
    

,0,1
0,a,1
1,b,2
2,<NA>,<NA>


In [54]:
cudfSeries = cudf.Series(['a1', 'b2', 'c3']).str.extract(r'([ab])(\d)')
print(cudfSeries)    

      0     1
0     a     1
1     b     2
2  <NA>  <NA>


Elements that do not match return a row filled with NaN. Thus, a Series of messy strings can be “converted” into a like-indexed Series or DataFrame of cleaned-up or more useful strings, without necessitating get() to access tuples or re.match objects. The dtype of the result is always object, even if no match is found and the result only contains NaN.

Named groups like

In [55]:
pd.Series(["a1", "b2", "c3"], dtype="string").str.extract(r"(?P<letter>[ab])(?P<digit>\d)", expand=False)
    

,letter,digit
0,a,1
1,b,2
2,<NA>,<NA>


Extracting a regular expression with one group returns a DataFrame with one column if expand=True.

In [56]:
pd.Series(["a1", "b2", "c3"], dtype="string").str.extract(r"[ab](\d)", expand=True)

,0
0,1
1,2
2,<NA>


In [57]:
cudf.Series(["a1", "b2", "c3"], dtype="str").str.extract(r"[ab](\d)", expand=True)

,0
0,1
1,2
2,<NA>


It returns a Series if expand=False.

In [58]:
pd.Series(["a1", "b2", "c3"], dtype="string").str.extract(r"[ab](\d)", expand=False)

0       1
1       2
2    <NA>
dtype: string

In [59]:
cudf.Series(["a1", "b2", "c3"], dtype="str").str.extract(r"[ab](\d)", expand=False)

0       1
1       2
2    <NA>
dtype: object

When each subject string in the Series has exactly one match,

In [60]:
pandasSeries = pd.Series(["a3", "b3", "c2"], dtype="string")

In [61]:
cudfSeries = cudf.Series(["a3", "b3", "c2"], dtype="str")


## Testing for strings that match or contain a pattern

You can check whether elements contain a pattern:

In [62]:
pattern = r"[0-9][a-z]"

pd.Series(["1", "2", "3a", "3b", "03c", "4dx"],dtype="str",
         ).str.contains(pattern)
   

0    False
1    False
2     True
3     True
4     True
5     True
dtype: bool

In [63]:
pattern = r"[0-9][a-z]"

cudf.Series(["1", "2", "3a", "3b", "03c", "4dx"],dtype="str",
         ).str.contains(pattern)
   

0    False
1    False
2     True
3     True
4     True
5     True
dtype: bool

Or whether elements match a pattern:

In [64]:
pd.Series(["1", "2", "3a", "3b", "03c", "4dx"],dtype="string",
         ).str.match(pattern)
   

0    False
1    False
2     True
3     True
4    False
5     True
dtype: boolean

In [65]:
cudf.Series(["1", "2", "3a", "3b", "03c", "4dx"],dtype="str",
         ).str.match(pattern) 

0    False
1    False
2     True
3     True
4    False
5     True
dtype: bool

New in version 1.1.0.

In [66]:
pd.Series(["1", "2", "3a", "3b", "03c", "4dx"],dtype="string",
         ).str.fullmatch(pattern)
    

0    False
1    False
2     True
3     True
4    False
5    False
dtype: boolean

In [67]:
cudf.Series(["1", "2", "3a", "3b", "03c", "4dx"],dtype="str",
         ).str.match(pattern)

0    False
1    False
2     True
3     True
4    False
5     True
dtype: bool

Methods like match, fullmatch, contains, startswith, and endswith take an extra na argument so missing values can be considered True or False:

In [68]:
pandasSeries5 = pd.Series(["A", "B", "C", "Aaba", "Baca", np.nan, "CABA", "dog", "cat"], dtype="string")   

pandasSeries5.str.contains("A", na=False)

print(pandasSeries5.str.swapcase())
print("Strings that start with 'b':")
print(pandasSeries5.str.startswith ('b'))
print(("Strings that ends with 'a':"))
print(pandasSeries5.str.endswith ('a'))

0       a
1       b
2       c
3    aABA
4    bACA
5    <NA>
6    caba
7     DOG
8     CAT
dtype: string
Strings that start with 'b':
0    False
1    False
2    False
3    False
4    False
5     <NA>
6    False
7    False
8    False
dtype: boolean
Strings that ends with 'a':
0    False
1    False
2    False
3     True
4     True
5     <NA>
6    False
7    False
8    False
dtype: boolean


In [69]:
cudfSeries5 = cudf.Series(["A", "B", "C", "Aaba", "Baca", np.nan, "CABA", "dog", "cat"], dtype="str")   

cudfSeries5.str.contains("A")

print(cudfSeries5.str.swapcase())
print(cudfSeries5.str.swapcase())
print("Strings that start with 'b':")
print(cudfSeries5.str.startswith ('b'))
print(("Strings that ends with 'a':"))
print(cudfSeries5.str.endswith ('a'))

0       a
1       b
2       c
3    aABA
4    bACA
5    <NA>
6    caba
7     DOG
8     CAT
dtype: object
0       a
1       b
2       c
3    aABA
4    bACA
5    <NA>
6    caba
7     DOG
8     CAT
dtype: object
Strings that start with 'b':
0    False
1    False
2    False
3    False
4    False
5     <NA>
6    False
7    False
8    False
dtype: bool
Strings that ends with 'a':
0    False
1    False
2    False
3     True
4     True
5     <NA>
6    False
7    False
8    False
dtype: bool


In [70]:
cudfSeries5 = cudf.Series(["A", "B", "C", "Aaba", "Baca", np.nan, "CABA", "dog", "cat"], dtype="str")   


